In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/media/beans/ssd/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'1ewnimd2'

In [3]:
wandb.init(id='1ewnimd2', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

13502.597

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=3e-5)

In [7]:
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.48 µs


In [9]:
#wandb.watch(m, log="all")

In [10]:
#train_only_parts_of_model(m, part_to_train="fcs_2")

In [11]:
model_stem = "7.8"
START_E = 0

In [ ]:
n_epochs = 100
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = np.interp(e, [0, 10, 20, 30], [3e-4, 1e-4, 1e-4, 3e-5])
    
    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 0

{'trn_control_loss': 0.02162781, 'trn_headings_loss': 0.04558086, 'consistency losses/trn_steer_cost': 0.01160961, 'consistency losses/trn_te_loss': 15.55573067, 'consistency losses/trn_torque_delta_loss': 32.89880321, 'logistical/obs_consumed_per_second': 237.83464567, 'logistical/obs_generated_per_second': 108.84251969, 'logistical/data_consumption_ratio': 2.18768086, 'logistical/manual_train_pause': 0.0, 'consistency losses/trn_torque_loss': 82.21726699, 'logistical/max_param': 16.24296951, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 235.6015625, 'logistical/obs_generated_per_second': 113.140625, 'logistical/data_consumption_ratio': 2.08549804, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0095039, 'trn_headings_loss': 0.02780467, 'consistency losses/trn_steer_cost': 0.0117321, 'consistency losses/trn_te_loss': 15.73619703, 'consistency losses/trn_torque_delta_loss': 33.15937065, 'consistency losses/trn_torque_loss': 77.38202858, 'logistica

In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e29.torch")


/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
